私たちのクライアントは、顧客に健康保険を提供している保険会社です。
顧客は、昨年の保険契約者（顧客）が、会社の提供する自動車保険にも関心があるかどうかを予測するモデルを構築するためにあなたの助けを必要としています。

保険契約とは、会社が特定の保険料の支払いと引き換えに、特定の損失、損害、病気、または死亡に対する補償の保証を提供することを約束する取り決めです。
保険料は、この保証のために顧客が保険会社に定期的に支払う必要のある金額の合計です。

たとえば、健康保険のカバーのために毎年5000ルピー支払います。
もしもの場合、あなたが病気になったり、その年に入院する必要がある場合、保険会社は最大Rs200000ルピーの入院費用などを負担します。
今、会社がどのようにそのような高い入院費用を負担することができるか、あなたが疑問に思っているならば、ここで確率の概念が浮かび上がります。

たとえば、あなたのように、ルピーのプレミアムを支払う顧客が100人いるとします。
毎年5000人ですが、その年に入院するのはごくわずか（たとえば2〜3人）で、全員ではありません。
このようにして、誰もが他のすべての人のリスクを共有します。

医療保険と同様に、毎年一定額の保険料を保険会社に支払わなければならない自動車保険があります。
車両による不幸な事故が発生した場合、保険会社が補償（「保証額」と呼ばれる）を顧客へ提供します。

顧客が自動車保険に関心があるかどうかを予測するモデルを構築することは、企業にとって非常に役立ちます。
これにより、顧客に連絡し、ビジネスモデルと収益を最適化するためのコミュニケーション戦略を計画できるからです。

ここで、顧客が自動車保険に関心があるかどうかを予測するために、人口統計（性別、年齢、地域コードタイプ）、車両（車両の年齢、損傷）、ポリシー（プレミアム、調達チャネル）などに関する情報があります。



✔️データの説明

・id＝顧客の一意のID

・性別＝お客様の性別

・年齢＝お客様の年齢

・運転免許証＝0：お客様はDLを持っていません、1：お客様はすでにDLを持っています

・Region_Code＝お客様の地域に固有のコード

・以前_Insured＝1：お客様はすでに自動車保険に加入しています。0：お客様は自動車保険に加入していません。

・Vehicle_Age＝車両の年齢

・Vehicle_Damage＝1：お客様が過去に車両を破損したことがあります。0：お客様は過去に車両に損傷を与えていません。

・Annual_Premium＝顧客がその年に保険料として支払う必要のある金額

・ポリシー販売チャネル＝顧客へのアウトリーチのチャネルの匿名化されたコード。さまざまなエージェント、メール、電話、対面など。

・ビンテージ＝顧客が会社に関連付けられている日数

・応答＝1：お客様が興味を持っている、0：お客様が興味を持っていない



✔️提出

・id＝顧客の一意のID

・応答＝1：お客様が興味を持っている、0：お客様が興味を持っていない

・評価指標：このハッカソンの評価指標はROC_AUCスコアです。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train=pd.read_csv('../input/health-insurance-cross-sell-prediction/train.csv')
test=pd.read_csv('../input/health-insurance-cross-sell-prediction/test.csv')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
print(train.shape)
print(train.isnull().sum())
print(train.dtypes)

In [ ]:
print(train['Vehicle_Age'].unique())

In [ ]:
train.groupby(['Vehicle_Age','Response'])['id'].count()

In [ ]:
print(train['Policy_Sales_Channel'].unique())

In [ ]:
train.groupby(['Policy_Sales_Channel','Response'])['id'].count()

In [ ]:
df=train.groupby(['Previously_Insured','Response'])['id'].count().to_frame().rename(columns={'id':'count'}).reset_index()
df

In [ ]:
sns.catplot(x="Previously_Insured", y="count",col="Response",data=df, kind="bar",height=6, aspect=.7);
# すでに加入している人は、ほとんど興味を示さない

In [ ]:
train.describe()

In [ ]:
train.Response.value_counts()
# 興味ある率は１２．3％くらい

In [ ]:
sns.distplot(train.Age)
# 顧客は若い人が多い

In [ ]:
sns.countplot(train.Gender)

In [ ]:
sns.countplot(train.Driving_License)
# 免許はほぼ持っている

In [ ]:
sns.countplot(train.Previously_Insured)
# 自動車保険未加入の人が多い

In [ ]:
sns.countplot(train.Vehicle_Damage)
# 過去の事故や故障があった人は半分くらい

In [ ]:
sns.countplot(train.Response)

In [ ]:
sns.countplot(train.Vehicle_Age)
# ２年以内の人が大多数

In [ ]:
df２=train.groupby(['Vehicle_Age','Response'])['id'].count().to_frame().rename(columns={'id':'count'}).reset_index()
df２

In [ ]:
sns.catplot(x="Vehicle_Age", y="count",col="Response",data=df２, kind="bar",height=6, aspect=.7);
# １年以上２年未満の人の方が興味がある！

In [ ]:
df３ = train.groupby(['Previously_Insured','Vehicle_Damage'])['id'].count().to_frame().rename(columns={'id':'count'}).reset_index()
df3

In [ ]:
sns.catplot(x="Previously_Insured", y="count",col="Vehicle_Damage",data=df3, kind="bar",height=6, aspect=.7);
# ??? 過去に車両破損経験者の方が自動車保険に入ってない。
# 自動車保険に入っている人の方が車両破損しない。→安全運転、慎重？

In [ ]:
train.plot(kind='scatter', x='Age', y='Annual_Premium', figsize=(10, 7))
# 年齢とプレミアム保険料を払っている額

In [ ]:
train.groupby('Response')['Age'].plot.hist(bins=20, alpha=0.5, legend=True)
# 興味ある人の年齢の分布：40代付近が多い。若い人は興味ない。

In [ ]:
(ax0, ax1), = pd.crosstab(pd.cut(train['Age'], range(0, 101, 5), right=False), train['Response']).plot.barh(subplots=True, layout=(1, 2), sharex=False)
ax0.invert_xaxis() # 左側のグラフのX軸を反転する
ax1.set_yticklabels([]) # 右側のグラフのY軸のラベルを消す

In [ ]:
(ax0, ax1), = pd.crosstab(pd.cut(train['Annual_Premium'], range(0, 60000, 5000), right=False), train['Response']).plot.barh(subplots=True, layout=(1, 2), sharex=False)
ax0.invert_xaxis() # 左側のグラフのX軸を反転する
ax1.set_yticklabels([]) # 右側のグラフのY軸のラベルを消す

In [ ]:
sns.boxplot( x=train['Gender'], y=train['Age'] )
# 性別と年齢

In [ ]:
sns.boxplot( x=train['Vehicle_Age'], y=train['Age'] )
# 車両年数と年齢
# 買ったばかりの１年未満は20歳代が多い
# 買って１年以上2年未満は40歳50歳代が多い＝乗り換え？

In [ ]:
sns.boxplot( x=train['Response'], y=train['Age'] )
# 興味ある人と年齢

In [ ]:
sns.boxplot( x=train['Response'], y=train['Annual_Premium'] )
# 興味ある人と年間プレミアム保険料支払い者：あまり関係ない？

In [ ]:
train.loc[train['Gender'] == 'Male', 'Gender'] = 1
train.loc[train['Gender'] == 'Female', 'Gender'] = 0
test.loc[test['Gender'] == 'Male', 'Gender'] = 1
test.loc[test['Gender'] == 'Female', 'Gender'] = 0

train.loc[train['Vehicle_Age'] == '> 2 Years', 'Vehicle_Age'] = 2
train.loc[train['Vehicle_Age'] == '1-2 Year', 'Vehicle_Age'] = 1
train.loc[train['Vehicle_Age'] == '< 1 Year', 'Vehicle_Age'] = 0
test.loc[test['Vehicle_Age'] == '> 2 Years', 'Vehicle_Age'] = 2
test.loc[test['Vehicle_Age'] == '1-2 Year', 'Vehicle_Age'] = 1
test.loc[test['Vehicle_Age'] == '< 1 Year', 'Vehicle_Age'] = 0

train.loc[train['Vehicle_Damage'] == 'Yes', 'Vehicle_Damage'] = 1
train.loc[train['Vehicle_Damage'] == 'No', 'Vehicle_Damage'] = 0
test.loc[test['Vehicle_Damage'] == 'Yes', 'Vehicle_Damage'] = 1
test.loc[test['Vehicle_Damage'] == 'No', 'Vehicle_Damage'] = 0

In [ ]:
for col in train.columns:
    train[col] = train[col].astype(np.int32)

train.head(10)

In [ ]:
train_corr = train.corr()
matplotlib.pyplot.figure(figsize=(10,10))
sns.heatmap(train_corr, vmax=.9, square=True, annot=True, linewidths=.3, cmap="YlGnBu", fmt='.３f')

In [ ]:
train.plot(kind='scatter', x='Age', y='Policy_Sales_Channel', figsize=(10, 7))
# 流入経路は年齢との相関が強い
# 40−50、80−120あたりが40歳50歳代を獲得できるチャネルコード

Modeling してみましょう

In [ ]:
x = train.drop("Response",axis=1)
y = train["Response"]

In [ ]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
dt_clf = DecisionTreeClassifier(criterion='entropy',max_depth = 100,random_state=0)
dt_clf.fit(x_train,y_train)

In [ ]:
from sklearn.metrics import accuracy_score,classification_report
dt_pred = dt_clf.predict(x_test)
dt_accuracy = accuracy_score(y_test,dt_pred)
dt_accuracy

In [ ]:
rf_clf = RandomForestClassifier(n_estimators = 200,random_state=0)
rf_clf.fit(x_train,y_train)

In [ ]:
rf_pred = rf_clf.predict(x_test)
rf_accuracy = accuracy_score(y_test,rf_pred)
rf_accuracy

In [ ]:
lr_clf = LogisticRegression(random_state=0)
lr_clf.fit(x_train,y_train)

In [ ]:
lr_pred = lr_clf.predict(x_test)
lr_accuracy = accuracy_score(y_test,lr_pred)
lr_accuracy

In [ ]:
lgbm_clf = LGBMClassifier(n_estimators=1000,learning_rate=0.007,random_state=0)#1000
lgbm_clf.fit(x_train,y_train)

In [ ]:
lgbm_pred = lgbm_clf.predict(x_test)
lgbm_accuracy = accuracy_score(y_test,lgbm_pred)
lgbm_accuracy

In [ ]:
knn_clf = KNeighborsClassifier(n_neighbors=20)
knn_clf.fit(x_train,y_train)

In [ ]:
knn_pred = knn_clf.predict(x_test)
knn_accuracy = accuracy_score(y_test,knn_pred)
knn_accuracy

In [ ]:
acc_df = pd.DataFrame({"Decision Tree":dt_accuracy,"Random Forest":rf_accuracy,
                       "LightGBM":lgbm_accuracy,"Logistic Regression" : lr_accuracy,"KNN":knn_accuracy},index = ["Accuracy"])
acc_df.style.background_gradient(cmap = "Reds")

ランダムフォレストで格納してみましょう

In [ ]:
features=['Gender','Age','Driving_License','Region_Code','Previously_Insured','Vehicle_Age','Vehicle_Damage','Annual_Premium',
'Policy_Sales_Channel','Vintage']
target = 'Response'

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model_rfc = RandomForestClassifier(n_estimators=100,random_state=9,n_jobs=-1)

model_rfc.fit(train[features],train[target]) 

In [ ]:
predictions = model_rfc.predict(test[features])

predictions

In [ ]:
submission = pd.DataFrame({'id':test['id'],'Response':predictions})

submission.head(10)

In [ ]:
filename = 'submission.csv'

submission.to_csv(filename,index=False)

print('Saved file: ' + filename)